Importação de bibliotecas para abrir arquivos e medir o tempo de execução, além do multithreading

In [106]:
import time
import random
import threading
import logging

In [118]:
# Gerar números aleatórios e salvar em um arquivo
def gerar_numeros_aleatorios(arquivo: str, quantidade:int) -> None:
    """
    Gera uma quantidade especificada de números aleatórios e salva em um arquivo.
    """
    numeros = [str(random.randint(1, 10000)) for _ in range(quantidade)]
    with open(arquivo, 'w') as f:
        f.write(','.join(numeros))  # Usar espaço como separador
    print(f"Números aleatórios gerados e salvos em {arquivo}")

In [124]:
gerar_numeros_aleatorios('../Arquivos/input/numerosSimples.txt', 10000000)  # Gerar 100 números aleatórios e salvar no arquivo

Números aleatórios gerados e salvos em ../Arquivos/input/numerosSimples.txt


### Implementação Insertion Sort
* O Insertion Sort é um algoritmo de ordenação que constrói a lista ordenada um elemento de cada vez.
* Ele é mais eficiente para listas pequenas ou listas que já estão quase ordenadas.
* O algoritmo funciona dividindo a lista em duas partes: a parte ordenada e a parte não ordenada.
* Ele pega um elemento da parte não ordenada e o insere na posição correta na parte ordenada.
* Varia entre O(n^2) e O(n) dependendo da ordenação inicial da lista.

In [109]:
def insertion_sort(lista: list) -> list:
    """
    Ordena uma lista usando o algoritmo Insertion Sort.

    Args:
        lista: A lista a ser ordenada.

    Returns:
        A lista ordenada.
    """
    # Verificar se a lista já está ordenada
    for i in range(1, len(lista)):
        chave = lista[i]
        j = i - 1
        while j >= 0 and lista[j] > chave:
            lista[j + 1] = lista[j]
            j -= 1
        lista[j + 1] = chave
    return lista

In [110]:
def insertion_sort_multithreaded(lista: list, num_threads:int,logger) -> list:
    """
    Ordena uma lista usando o algoritmo Insertion Sort com múltiplas threads.

    Args:
        lista: A lista a ser ordenada.
        num_threads: O número de threads a serem usadas.

    Returns:
        A lista ordenada.
    """
    if num_threads <= 0:
        logger.error("O número de threads deve ser maior que zero.")
        raise ValueError("O número de threads deve ser maior que zero.")
    logger.info(f"Iniciando Insertion Sort com {num_threads} threads.")
    tamanho = len(lista)
    threads = []
    partes = [lista[i * tamanho // num_threads:(i + 1) * tamanho // num_threads] for i in range(num_threads)]

    for i in range(num_threads):
        thread = threading.Thread(target=insertion_sort, args=(partes[i],))
        threads.append(thread)
        thread.start()
        #Printar informações sobre a thread
        logger.info(f"Thread {i + 1} iniciada para ordenar a parte {i + 1} com {len(partes[i])} elementos.")
    logger.info("Threads iniciadas para ordenação.")
    # Aguardar todas as threads terminarem

    for thread in threads:
        thread.join()
    logger.info("Todas as threads concluídas.")

    # Combinar as partes ordenadas
    resultado = []
    for parte in partes:
        resultado.extend(parte)
    logger.info("Partes combinadas após ordenação.")

    return insertion_sort(resultado)  # Ordenar a lista combinada


### Bucketsort
* Não é um método de ordenação, e sim um método de distribuição.
* O Bucket Sort é um algoritmo de ordenação que distribui os elementos em vários "baldes" (buckets).
* Cada balde é ordenado individualmente, geralmente usando um algoritmo de ordenação simples, como o Insertion Sort.
* O Bucket Sort é eficiente para listas com valores uniformemente distribuídos.
* Por exemplo, se tivermos uma lista entre 0 e 25, podemos criar 5 baldes: [0-5], [6-10], [11-15], [16-20], [20-25].
    * Para dividir os elementos, podemos usar a fórmula: bucket = elemento / 5
    * Arredondamos para cima, pois o bucket 0 é o [0-5], o bucket 1 é o [6-10], e assim por diante.
* Após a distribuição, ordenamos cada balde individualmente.
* Após a ordenação, concatenamos os baldes para obter a lista ordenada.

In [111]:
# Método Bucket Sort para auxiliar na ordenação
def separar_buckets(lista: list) -> list:
    """
    Separa os elementos da lista em buckets.

    Args:
        lista: A lista a ser separada.

    Returns:
        A quantidade de buckets e a lista de buckets.
    """
    max_valor = max(lista)
    min_valor = min(lista)
    intervalo = (max_valor - min_valor) / len(lista)
    buckets = [[] for _ in range(len(lista))]

    for numero in lista:
        indice_bucket = int((numero - min_valor) / intervalo)
        if indice_bucket == len(buckets):  # Garantir que o último bucket não exceda o tamanho
            indice_bucket -= 1
        buckets[indice_bucket].append(numero)

    return buckets
    
def bucket_sort(lista: list) -> list:
    """
    Ordena uma lista usando o algoritmo Bucket Sort.

    Args:
        lista: A lista a ser ordenada.

    Returns:
        A lista ordenada.
    """
    if len(lista) == 0:
        return lista

    # Separar os elementos em buckets
    buckets = separar_buckets(lista)

    # Ordenar cada bucket individualmente
    for i in range(len(buckets)):
        buckets[i] = insertion_sort(buckets[i])

    # Concatenar os buckets ordenados
    resultado = []
    for bucket in buckets:
        resultado.extend(bucket)

    return resultado
  

In [112]:
def bucket_sort_multithreaded(lista: list, num_threads: int, logger) -> list:
    """
    Ordena uma lista usando o algoritmo Bucket Sort com múltiplas threads.

    Args:
        lista: A lista a ser ordenada.
        num_threads: O número de threads a serem usadas.
        logger: Logger para registrar informações.

    Returns:
        A lista ordenada.
    """
    if len(lista) == 0:
        return lista

    if num_threads <= 0:
        logger.error("O número de threads deve ser maior que zero.")
        raise ValueError("O número de threads deve ser maior que zero.")
        
    logger.info(f"Iniciando Bucket Sort com {num_threads} threads.")
    
    # Separar os elementos em buckets
    buckets = separar_buckets(lista)
    logger.info(f"Separados {len(buckets)} buckets para ordenação.")
    
    # Dividir os buckets entre as threads
    def ordenar_grupo_buckets(buckets_grupo):
        for bucket in buckets_grupo:
            insertion_sort(bucket)
    
    # Dividir os buckets em grupos para as threads
    buckets_por_thread = [[] for _ in range(num_threads)]
    for i, bucket in enumerate(buckets):
        buckets_por_thread[i % num_threads].append(bucket)
    
    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=ordenar_grupo_buckets, args=(buckets_por_thread[i],))
        threads.append(thread)
        thread.start()
        logger.info(f"Thread {i + 1} iniciada para ordenar {len(buckets_por_thread[i])} buckets.")
    
    for thread in threads:
        thread.join()
    logger.info("Todas as threads concluídas.")

    # Concatenar os buckets ordenados
    resultado = []
    for bucket in buckets:
        resultado.extend(bucket)
    logger.info("Buckets combinados após ordenação.")

    return resultado

In [113]:
# Abrir o arquivo "Numeros.txt" e ler os números
def abrir_arquivo(arquivo: str) -> list:
    """
    Lê os números de um arquivo e retorna uma lista.

    Args:
        arquivo: O caminho do arquivo a ser lido.

    Returns:
        Uma lista com os números lidos do arquivo.
    """
    with open(arquivo, 'r') as f:
        # Os numeros estão separados por vírgula
        numeros = f.read().strip().split(',')
    # Converter os números de string para inteiro
    numeros = [int(num) for num in numeros if num.isdigit()]
    return numeros

In [127]:
# Função para criar o arquivo de saída com os números ordenados, o tempo de execução e o método (Insertion Sort com ou sem bucketsort) e técnica de threading
def criar_arquivo_saida(arquivo_output:str, worA:str, num_elementos:int, tempo: float, metodo: str, multiThreading:bool)-> None:
    """
    Cria um arquivo de saída com os números ordenados, o tempo de execução e o método utilizado.

    Args:
        numeros: A lista de números ordenados.
        tempo: O tempo de execução do algoritmo.
        metodo: O método utilizado para ordenar os números.
    """
    with open(arquivo_output, worA) as f:
        f.write('Quantidade de Elementos,Tempo de Execução (s),Método,Multithreading\n')
        f.write(f'{num_elementos},{tempo},{metodo},{"Sim" if multiThreading else "Nao"}\n')

In [128]:
def executar_ordenacao(algoritmo: str, usar_threads: bool = False, num_threads: int = 4, 
                       arquivo_entrada: str = '../Arquivos/input/numerosSimples.txt', 
                       arquivo_saida: str = '../Arquivos/output/outputSimples.csv',
                       modo_escrita: str = 'a'):
    """
    Função generalizada para executar algoritmos de ordenação com ou sem multithreading.
    
    Args:
        algoritmo: O algoritmo de ordenação a ser usado ('insertion' ou 'bucket').
        usar_threads: Se deve usar multithreading (True) ou não (False).
        num_threads: O número de threads a serem usadas se usar_threads for True.
        arquivo_entrada: O caminho do arquivo de entrada.
        arquivo_saida: O caminho do arquivo de saída.
        modo_escrita: O modo de escrita do arquivo ('a' para append ou 'w' para write).
    """
    # Configurar logger
    logger = logging.getLogger(f"{algoritmo}_{usar_threads}")
    logger.setLevel(logging.INFO)
    # Verificar se já existe handler para evitar duplicação de logs
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    
    # Abrir o arquivo e ler os números
    numeros = abrir_arquivo(arquivo_entrada)
    
    print(f"Números lidos do arquivo: {len(numeros)} números")
    
    # Medir o tempo de execução do algoritmo
    inicio = time.time()
    
    # Escolher o algoritmo e se deve usar multithreading
    if algoritmo.lower() == 'insertion':
        if usar_threads:
            logger.info(f"Usando {num_threads} threads para Insertion Sort.")
            numeros_ordenados = insertion_sort_multithreaded(numeros.copy(), num_threads, logger)
            metodo = 'Insertion Sort'
        else:
            logger.info("Usando Insertion Sort sem multithreading.")
            numeros_ordenados = insertion_sort(numeros.copy())
            metodo = 'Insertion Sort'
    elif algoritmo.lower() == 'bucket':
        if usar_threads:
            logger.info(f"Usando {num_threads} threads para Bucket Sort.")
            numeros_ordenados = bucket_sort_multithreaded(numeros.copy(), num_threads, logger)
            metodo = 'Bucket Sort'
        else:
            logger.info("Usando Bucket Sort sem multithreading.")
            numeros_ordenados = bucket_sort(numeros.copy())
            metodo = 'Bucket Sort'
    else:
        logger.error(f"Algoritmo {algoritmo} não reconhecido.")
        raise ValueError(f"Algoritmo {algoritmo} não reconhecido. Use 'insertion' ou 'bucket'.")
    
    fim = time.time()
    
    # Calcular o tempo de execução
    tempo_execucao = fim - inicio
    print(f"Tempo de execução: {tempo_execucao:.6f} segundos")
    
    # Criar o arquivo de saída com os resultados
    criar_arquivo_saida(arquivo_saida, modo_escrita,len(numeros), tempo_execucao, metodo, usar_threads)
    
    return numeros_ordenados

In [120]:
# Criar um lock global
ordenacao_lock = threading.Lock()
def executar_todos_metodos(arquivo_entrada='../Arquivos/input/numerosSimples.txt', 
                          arquivo_saida='../Arquivos/output/outputBucketSimples.csv'):
    """
    Executa todos os métodos de ordenação e salva os resultados.
    Usa um lock para garantir que apenas um método seja executado por vez.
    
    Args:
        arquivo_entrada: O caminho do arquivo de entrada.
        arquivo_saida: O caminho do arquivo de saída.
    """
    # Adquirir o lock global para garantir execução exclusiva
    with ordenacao_lock:
        print("Iniciando execução de todos os métodos de ordenação...")
        
        # Insertion Sort sem threads
        # print("\n=== Executando Insertion Sort sem threads ===")
        # executar_ordenacao('insertion', False, arquivo_entrada=arquivo_entrada, 
        #                    arquivo_saida=arquivo_saida, modo_escrita='w')
        
        # Bucket Sort sem threads
        print("\n=== Executando Bucket Sort sem threads ===")
        executar_ordenacao('bucket', False, arquivo_entrada=arquivo_entrada, 
                          arquivo_saida=arquivo_saida)
        
        # Insertion Sort com threads
        # print("\n=== Executando Insertion Sort com threads ===")
        # executar_ordenacao('insertion', True, arquivo_entrada=arquivo_entrada, 
        #                    arquivo_saida=arquivo_saida)
        
        # Bucket Sort com threads
        print("\n=== Executando Bucket Sort com threads ===")
        executar_ordenacao('bucket', True, arquivo_entrada=arquivo_entrada, 
                          arquivo_saida=arquivo_saida)
        
        print("\nTodos os métodos de ordenação foram executados com sucesso.")

In [129]:
executar_todos_metodos()

Iniciando execução de todos os métodos de ordenação...

=== Executando Bucket Sort sem threads ===


2025-06-18 11:21:50,888 - bucket_False - INFO - Usando Bucket Sort sem multithreading.


Números lidos do arquivo: 10000000 números
Tempo de execução: 20.697562 segundos

=== Executando Bucket Sort com threads ===


2025-06-18 11:22:15,445 - bucket_True - INFO - Usando 4 threads para Bucket Sort.
2025-06-18 11:22:15,519 - bucket_True - INFO - Iniciando Bucket Sort com 4 threads.


Números lidos do arquivo: 10000000 números


2025-06-18 11:22:28,542 - bucket_True - INFO - Separados 10000000 buckets para ordenação.
2025-06-18 11:22:29,874 - bucket_True - INFO - Thread 1 iniciada para ordenar 2500000 buckets.
2025-06-18 11:22:30,048 - bucket_True - INFO - Thread 2 iniciada para ordenar 2500000 buckets.
2025-06-18 11:22:30,540 - bucket_True - INFO - Thread 3 iniciada para ordenar 2500000 buckets.
2025-06-18 11:22:31,264 - bucket_True - INFO - Thread 4 iniciada para ordenar 2500000 buckets.
2025-06-18 11:22:35,971 - bucket_True - INFO - Todas as threads concluídas.
2025-06-18 11:22:36,971 - bucket_True - INFO - Buckets combinados após ordenação.


Tempo de execução: 22.277438 segundos

Todos os métodos de ordenação foram executados com sucesso.
